<a href="https://colab.research.google.com/github/John4088/NTHU_2024_DLBOI_HW/blob/main/hw4_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Task A: Model Selection
# Step 1: 安裝所需的 pytorch 和 torchvision 套件
!pip install torch torchvision

# Step 2: 導入所需的庫
import torch
import torchvision.models as models
from time import time

# Step 3: 選擇兩個預訓練模型
# 這裡選擇了 ResNet50 和 VGG16 作為遷移學習的模型
model1_name = "ResNet50"
model2_name = "VGG16"
model1 = models.resnet50(pretrained=True)
model2 = models.vgg16(pretrained=True)

# 顯示模型架構
print(f"{model1_name} 架構:\n", model1)
print(f"\n{model2_name} 架構:\n", model2)

# Step 4: 模型選擇理由
model_choices = {
    model1_name: {
        "architecture": "ResNet50 使用殘差塊結構，適合處理更深的網路，並可以降低梯度消失的風險。",
        "pretrained_performance": "在 ImageNet 上有高準確度，適合複雜的特徵提取。",
        "transfer_learning_potential": "因為架構較深且適應性強，適合遷移學習中的細微分類任務。"
    },
    model2_name: {
        "architecture": "VGG16 使用多層卷積層，結構相對簡單，易於理解和修改。",
        "pretrained_performance": "在 ImageNet 上也具備良好準確度，尤其在圖片分類方面。",
        "transfer_learning_potential": "穩定適用於遷移學習，適合計算時間較短的任務，但可能需要調整層數以適應新任務。"
    }
}

# 顯示模型選擇理由
for model_name, info in model_choices.items():
    print(f"\n{model_name} 選擇理由：")
    for key, value in info.items():
        print(f"{key}: {value}")

# Step 5: 測試推理時間
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_tensor = torch.rand(1, 3, 224, 224).to(device)  # 隨機生成一個測試圖像

def measure_time(model, input_tensor):
    model = model.to(device)  # 將模型轉移到 GPU（如果可用）
    model.eval()  # 設置模型為評估模式
    with torch.no_grad():
        # 測量推理時間
        start_time = time()
        model(input_tensor)
        end_time = time()
        return (end_time - start_time) * 1000  # 轉換為毫秒

# 測量兩個模型的推理時間
print("\n推理時間測試（毫秒）：")
print(f"{model1_name}:", measure_time(model1, input_tensor))
print(f"{model2_name}:", measure_time(model2, input_tensor))


ResNet50 架構:
 ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1

In [ ]:
# Task B: Fine-tuning the ConvNet
# Step 1: 安裝所需的套件
!pip install torch torchvision

# Step 2: 導入所需的庫
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score

# Step 3: 資料預處理與資料集加載
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 使用 FakeData 模擬二分類 X 光資料集（以 1000 張訓練圖像和 200 張測試圖像為例）
train_dataset = datasets.FakeData(size=1000, transform=transform, num_classes=2)
test_dataset = datasets.FakeData(size=200, transform=transform, num_classes=2)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 4: 模型選擇與微調設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 選擇 ResNet50 和 VGG16 並修改最後一層以適應二分類
model1 = models.resnet50(pretrained=True)
model1.fc = nn.Linear(model1.fc.in_features, 2)  # 修改 ResNet50 的全連接層
model1 = model1.to(device)

model2 = models.vgg16(pretrained=True)
model2.classifier[6] = nn.Linear(model2.classifier[6].in_features, 2)  # 修改 VGG16 的全連接層
model2 = model2.to(device)

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)

# Step 5: 訓練與微調函數
def train_model(model, optimizer, train_loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# 微調 ResNet50
print("微調 ResNet50...")
train_model(model1, optimizer1, train_loader)

# 微調 VGG16
print("\n微調 VGG16...")
train_model(model2, optimizer2, train_loader)

# Step 6: 性能評估函數
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# 評估微調後的模型
print("\n性能評估:")
resnet50_accuracy = evaluate_model(model1, test_loader)
vgg16_accuracy = evaluate_model(model2, test_loader)
print(f"ResNet50 測試集準確率: {resnet50_accuracy:.4f}")
print(f"VGG16 測試集準確率: {vgg16_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
 

微調 ResNet50...
Epoch [1/5], Loss: 0.7468
Epoch [2/5], Loss: 0.7081
Epoch [3/5], Loss: 0.7308
Epoch [4/5], Loss: 0.7029
Epoch [5/5], Loss: 0.6972

微調 VGG16...
Epoch [1/5], Loss: 1.5614
Epoch [2/5], Loss: 0.7147
Epoch [3/5], Loss: 0.7011
Epoch [4/5], Loss: 0.6972
Epoch [5/5], Loss: 0.7078

性能評估:
ResNet50 測試集準確率: 0.5450
VGG16 測試集準確率: 0.4600


In [ ]:
# Task C: ConvNet as Fixed Feature Extractor
# Step 1: 安裝所需的套件
!pip install torch torchvision

# Step 2: 導入所需的庫
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score

# Step 3: 資料預處理與資料集加載
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 使用 FakeData 模擬二分類 X 光資料集（以 1000 張訓練圖像和 200 張測試圖像為例）
train_dataset = datasets.FakeData(size=1000, transform=transform, num_classes=2)
test_dataset = datasets.FakeData(size=200, transform=transform, num_classes=2)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 4: 模型選擇與固定特徵提取器設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 選擇 ResNet50 和 VGG16 並凍結所有層（僅訓練最後一層）
model1 = models.resnet50(pretrained=True)
for param in model1.parameters():
    param.requires_grad = False
model1.fc = nn.Linear(model1.fc.in_features, 2)  # 修改最後一層
model1 = model1.to(device)

model2 = models.vgg16(pretrained=True)
for param in model2.parameters():
    param.requires_grad = False
model2.classifier[6] = nn.Linear(model2.classifier[6].in_features, 2)  # 修改最後一層
model2 = model2.to(device)

# 定義損失函數和優化器（只優化最後一層）
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.fc.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.classifier[6].parameters(), lr=0.001)

# Step 5: 訓練函數
def train_model_fixed_extractor(model, optimizer, train_loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# 訓練 ResNet50 作為固定特徵提取器
print("ResNet50 固定特徵提取器模式訓練...")
train_model_fixed_extractor(model1, optimizer1, train_loader)

# 訓練 VGG16 作為固定特徵提取器
print("\nVGG16 固定特徵提取器模式訓練...")
train_model_fixed_extractor(model2, optimizer2, train_loader)

# Step 6: 性能評估函數
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# 評估作為固定特徵提取器的模型性能
print("\n性能評估:")
resnet50_accuracy = evaluate_model(model1, test_loader)
vgg16_accuracy = evaluate_model(model2, test_loader)
print(f"ResNet50 測試集準確率: {resnet50_accuracy:.4f}")
print(f"VGG16 測試集準確率: {vgg16_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
 

ResNet50 固定特徵提取器模式訓練...
Epoch [1/5], Loss: 0.7388
Epoch [2/5], Loss: 0.7052
Epoch [3/5], Loss: 0.6719
Epoch [4/5], Loss: 0.6569
Epoch [5/5], Loss: 0.6350

VGG16 固定特徵提取器模式訓練...
Epoch [1/5], Loss: 0.7757
Epoch [2/5], Loss: 0.7414
Epoch [3/5], Loss: 0.7162
Epoch [4/5], Loss: 0.7440
Epoch [5/5], Loss: 0.7481

性能評估:
ResNet50 測試集準確率: 0.5000
VGG16 測試集準確率: 0.5400


In [ ]:
# Task D: Comparison and Analysis
# Step 1: 安裝所需的套件
!pip install torch torchvision

# Step 2: 導入所需的庫
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score

# Step 3: 資料預處理與資料集加載
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 使用 FakeData 模擬二分類 X 光資料集（以 1000 張訓練圖像和 200 張測試圖像為例）
train_dataset = datasets.FakeData(size=1000, transform=transform, num_classes=2)
test_dataset = datasets.FakeData(size=200, transform=transform, num_classes=2)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 4: 定義模型訓練和評估函數
def train_model(model, optimizer, train_loader, epochs=5):
    model.train()
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# Step 5: 微調模式下訓練和評估
def finetune_model():
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print("\n微調 ResNet50 模型訓練:")
    train_model(model, optimizer, train_loader)
    accuracy = evaluate_model(model, test_loader)
    print(f"微調模式 ResNet50 測試集準確率: {accuracy:.4f}")
    return accuracy

# Step 6: 固定特徵提取模式下訓練和評估
def feature_extractor_model():
    model = models.resnet50(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)
    optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

    print("\n固定特徵提取 ResNet50 模型訓練:")
    train_model(model, optimizer, train_loader)
    accuracy = evaluate_model(model, test_loader)
    print(f"固定特徵提取模式 ResNet50 測試集準確率: {accuracy:.4f}")
    return accuracy

# Step 7: 比較與分析
print("開始進行微調與固定特徵提取模型的比較:")
finetune_accuracy = finetune_model()
feature_extractor_accuracy = feature_extractor_model()

print("\n模型比較與分析結果:")
print(f"微調模式下 ResNet50 測試集準確率: {finetune_accuracy:.4f}")
print(f"固定特徵提取模式下 ResNet50 測試集準確率: {feature_extractor_accuracy:.4f}")

# 簡要分析
if finetune_accuracy > feature_extractor_accuracy:
    print("微調模式的性能更佳，說明模型在允許所有層進行更新時，能更好地適應新任務的細微特徵。")
else:
    print("固定特徵提取模式的性能更佳，顯示出預訓練特徵在此二分類任務上已經相當適合，不需要額外的微調。")


開始進行微調與固定特徵提取模型的比較:


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



微調 ResNet50 模型訓練:
Epoch [1/5], Loss: 0.7954
Epoch [2/5], Loss: 0.6922
Epoch [3/5], Loss: 0.6123
Epoch [4/5], Loss: 0.4028
Epoch [5/5], Loss: 0.2963
微調模式 ResNet50 測試集準確率: 0.9300


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



固定特徵提取 ResNet50 模型訓練:
Epoch [1/5], Loss: 0.7516
Epoch [2/5], Loss: 0.6968
Epoch [3/5], Loss: 0.6762
Epoch [4/5], Loss: 0.6933
Epoch [5/5], Loss: 0.6634
固定特徵提取模式 ResNet50 測試集準確率: 0.5550

模型比較與分析結果:
微調模式下 ResNet50 測試集準確率: 0.9300
固定特徵提取模式下 ResNet50 測試集準確率: 0.5550
微調模式的性能更佳，說明模型在允許所有層進行更新時，能更好地適應新任務的細微特徵。


In [ ]:
# Task E: Test Dataset Analysis
# Step 1: 安裝所需的套件
!pip install torch torchvision

# Step 2: 導入所需的庫
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score
import numpy as np

# Step 3: 資料預處理與資料集加載
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 使用 FakeData 模擬二分類 X 光資料集（以 1000 張訓練圖像和 200 張測試圖像為例）
train_dataset = datasets.FakeData(size=1000, transform=transform, num_classes=2)
test_dataset = datasets.FakeData(size=200, transform=transform, num_classes=2)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 4: 定義 ResNet50 模型，並僅調整最後一層以適應二分類任務
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # 修改最後一層輸出為二分類
model = model.to(device)

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: 訓練函數
def train_model(model, optimizer, train_loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Step 6: 評估函數
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# Step 7: 模型訓練與性能測試
print("模型訓練...")
train_model(model, optimizer, train_loader)

print("\n性能評估:")
train_accuracy = evaluate_model(model, train_loader)
test_accuracy = evaluate_model(model, test_loader)
print(f"訓練集準確率: {train_accuracy:.4f}")
print(f"測試集準確率: {test_accuracy:.4f}")

# Step 8: 分析測試資料集性能限制的原因
print("\n性能分析與討論:")
if test_accuracy < train_accuracy:
    print("在此實驗中，測試集性能低於訓練集，這可能是由以下幾個原因引起的：\n")
    print("1. 過擬合：模型在訓練集上過度學習，對訓練資料的特徵進行過度擬合，導致在測試集上泛化效果較差。")
    print("2. 測試資料集的分布與訓練資料集存在差異：即使 FakeData 模擬資料隨機生成，但在實際應用中，測試集可能包含不同分布或新特徵，導致模型無法正確分類。")
    print("3. 資料集大小限制：測試集樣本數較少，可能不足以代表真實的數據分布，導致性能測試時波動較大。")
    print("4. 模型架構限制：即便使用 ResNet50 等強大模型，當數據特徵複雜或需要特別處理時（例如 X 光影像中特徵差異不明顯），模型仍可能無法正確分類。")
else:
    print("測試集和訓練集的性能相當，模型具有良好的泛化性，但依然可能因資料集分布的細微差異而有所提升空間。")



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


模型訓練...
Epoch [1/5], Loss: 0.7576
Epoch [2/5], Loss: 0.6867
Epoch [3/5], Loss: 0.6751
Epoch [4/5], Loss: 0.4793
Epoch [5/5], Loss: 0.3751

性能評估:
訓練集準確率: 0.8550
測試集準確率: 0.8500

性能分析與討論:
在此實驗中，測試集性能低於訓練集，這可能是由以下幾個原因引起的：

1. 過擬合：模型在訓練集上過度學習，對訓練資料的特徵進行過度擬合，導致在測試集上泛化效果較差。
2. 測試資料集的分布與訓練資料集存在差異：即使 FakeData 模擬資料隨機生成，但在實際應用中，測試集可能包含不同分布或新特徵，導致模型無法正確分類。
3. 資料集大小限制：測試集樣本數較少，可能不足以代表真實的數據分布，導致性能測試時波動較大。
4. 模型架構限制：即便使用 ResNet50 等強大模型，當數據特徵複雜或需要特別處理時（例如 X 光影像中特徵差異不明顯），模型仍可能無法正確分類。
